In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from copy import copy
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

# 유틸 함수 import
from util import pre_time

# 데이터 로드
data = pd.read_csv('./data/직항_머신러닝.csv')
# 일본 데이터 추출
df = data.loc[data['partition_0']=='Japan']
# 시간 전처리
df = pre_time(df)

In [6]:
df.columns

Index(['arr_time', 'nights', 'dep_week', 'arr_week', 'agentcode',
       'partition_0', 'partition_1', 'time_zone', 'baggagetype', 'total_fare',
       'dep_jt', 'arr_jt', 'dep_time'],
      dtype='object')

In [7]:
df

,arr_time,nights,dep_week,arr_week,agentcode,partition_0,partition_1,time_zone,baggagetype,total_fare,dep_jt,arr_jt,dep_time
1459,4,3,7,INT005,Japan,FUK,Asia/Tokyo,F,309300,85,85,480,635
1460,4,3,7,INT005,Japan,FUK,Asia/Tokyo,F,375600,85,90,480,985
1461,4,3,7,INT005,Japan,FUK,Asia/Tokyo,F,406600,85,90,480,1265
1462,4,3,7,INT005,Japan,FUK,Asia/Tokyo,F,269400,85,85,1115,635
1463,4,3,7,INT005,Japan,FUK,Asia/Tokyo,F,335800,85,90,1115,985
...,...,...,...,...,...,...,...,...,...,...,...,...,...
714787,4,2,6,VGT027,Japan,TYO,Asia/Tokyo,F,373100,155,140,445,805
714788,4,2,6,VGT027,Japan,TYO,Asia/Tokyo,F,373100,155,150,445,1030
714789,4,2,6,VGT027,Japan,TYO,Asia/Tokyo,F,404200,155,160,445,620
714790,4,2,6,VGT027,Japan,TYO,Asia/Tokyo,F,382000,155,150,445,670


In [6]:
## agent 별 공항 분포

# print(df['partition_1'].value_counts())
# airport_list = df['partition_1'].unique().tolist()
# agent_partition_data = df[['agentcode','partition_1','partition_0']].groupby('agentcode').value_counts().reset_index()

# unique_partitions = df['partition_1'].unique()
# palette = sns.color_palette(palette='bright',n_colors= len(unique_partitions))
# color_mapping = dict(zip(unique_partitions, palette))


# for agent in df['agentcode'].unique():

#     plt.figure(figsize=(12, 6))
#     sns.barplot(x='partition_1', y='count',
#                 data=agent_partition_data.loc[agent_partition_data['agentcode']==agent],
#                 palette=color_mapping)
    
#     plt.xlabel('출발 시간 (분 단위)')
#     plt.ylabel('항공편 수')
#     plt.title(f'{agent} \n agent 별 공항 분포')
#     plt.xticks(rotation=90, size=8)
#     plt.tight_layout()

#     # 그래프 보여주기
#     plt.show()

In [11]:
df.groupby(['partition_1','dep_week','departure_time'])

,nights,departure_jt,dep_week,arrival_jt,arr_week,agentcode,partition_0,partition_1,time_zone,baggagetype,total_fare,departure_time,arrival_time
1459,4,85,3,85,7,INT005,Japan,FUK,Asia/Tokyo,F,309300,480,635
1460,4,85,3,90,7,INT005,Japan,FUK,Asia/Tokyo,F,375600,480,985
1461,4,85,3,90,7,INT005,Japan,FUK,Asia/Tokyo,F,406600,480,1265
1462,4,85,3,85,7,INT005,Japan,FUK,Asia/Tokyo,F,269400,1115,635
1463,4,85,3,90,7,INT005,Japan,FUK,Asia/Tokyo,F,335800,1115,985
...,...,...,...,...,...,...,...,...,...,...,...,...,...
714787,4,155,2,140,6,VGT027,Japan,TYO,Asia/Tokyo,F,373100,445,805
714788,4,155,2,150,6,VGT027,Japan,TYO,Asia/Tokyo,F,373100,445,1030
714789,4,155,2,160,6,VGT027,Japan,TYO,Asia/Tokyo,F,404200,445,620
714790,4,155,2,150,6,VGT027,Japan,TYO,Asia/Tokyo,F,382000,445,670
